# Example Decision Rule Calculations

This notebook provides simple example calculations for signallers in the disclosure game. The example follows the decision process of a light drinker who has played three rounds with a succession of particularly judgemental midwives, signalling honestly in two and claiming to be a moderate drinker in one.

In [1]:
import disclosuregame
from disclosuregame.Agents.bayes import BayesianSignaller, Responder
from disclosuregame.Agents.cpt import ProspectTheorySignaller
from disclosuregame.Agents.heuristic import SharingLexicographicSignaller
from disclosuregame.Agents.payoff import BayesianPayoffSignaller
from disclosuregame.Games.game import SimpleGame

In [9]:
response_alpha=[[1.0, 1.0], [1.0, 1.0], [1.0, 1.0]]
type_alpha=[1.0, 1.0, 1.0]

def make_agents(agent_fn):
    a = agent_fn(player_type=0, share_weight=1.0)
    b = Responder(player_type=1)
    game = SimpleGame(baby_payoff=10,referral_cost=9)
    a.init_payoffs(game.woman_baby_payoff, game.woman_social_payoff, response_weights=response_alpha, type_weights=type_alpha)
    return a, b

def play_games(agent, opponent):
    agent.log_signal(0)
    agent.update_counts(0, opponent, 10)
    agent.log_signal(0)
    agent.update_counts(0, opponent, 10)
    agent.log_signal(1)
    agent.update_counts(0, opponent, 9)

## Lexicographic

The lexicographic agent does a very simple counting process, and determines the most common outcomes of each signal (10 for light, 9 for moderate, and 8 for heavy). Since there is a clearly preferable option (light), it sends a light drinker signal.

In [10]:
a, b = make_agents(SharingLexicographicSignaller)
# Reset the payoff count, because init_payoffs counts 
# based on 'possible experiences' not literal alpha
for signal, payoffs in a.payoff_count.iteritems():
    for payoff, val in payoffs.iteritems():
        a.payoff_count[signal][payoff] = 1
play_games(a, b)
a.update_beliefs()
print "Most common outcome for light signal:", a.frequent(0, 0)
print "Most common outcome for moderate signal:", a.frequent(1, 0)
print "Most common outcome for heavy signal:", a.frequent(2, 0)
print "Signal choice is", a.signal_search([0, 1, 2])

0 <disclosuregame.Agents.bayes.Responder object at 0x10b682990> 10 None 1.0 0
Amending 1
Changed to 2.0
0 <disclosuregame.Agents.bayes.Responder object at 0x10b682990> 10 None 1.0 0
Amending 2.0
Changed to 3.0
0 <disclosuregame.Agents.bayes.Responder object at 0x10b682990> 9 None 1.0 1
Amending 1
Changed to 2.0
Most common outcome for light signal: 10
Most common outcome for moderate signal: 9
Most common outcome for heavy signal: 8
Signal choice is (0, 10)


## Payoff Based Bayesian Risk Minimisation

The payoff Bayesian risk minimisation agent takes a similar approach, but uses the liklihood of the outcomes in combination with the outcome, rather than just counting. 

The possible pairings of signals and outcomes are $[(l,10),(m,10),(h,10),(m,9),(h,9),(h,8)]$, with $\alpha_{i} = 1,n_i=0$ for all $i$. 
After playing the three rounds, $n_{l,10} = 2$, and $n_{m,9} = 1$.

In [16]:
a, b = make_agents(BayesianPayoffSignaller)
# Reset the payoff count, because init_payoffs counts 
# based on 'possible experiences' not literal alpha
for signal, payoffs in a.payoff_count.iteritems():
    for payoff, val in payoffs.iteritems():
        a.payoff_count[signal][payoff] = 1
print "Initial beliefs:"
print "Signal-action counts:",a.payoff_count
print "Prior beliefs:", a.payoff_belief
print "Risk attached to light signal:", a.risk(0)
print "Risk attached to moderate signal:", a.risk(1)
print "Risk attached to heavy signal:", a.risk(2)
print "Signal choice is", a.signal_search([0, 1, 2])
play_games(a, b)
a.update_beliefs()
print "After three games:"
print "Signal-action counts:",a.payoff_count
print "Beliefs:", a.payoff_belief
print "Risk attached to light signal:", a.risk(0)
print "Risk attached to moderate signal:", a.risk(1)
print "Risk attached to heavy signal:", a.risk(2)
print "Signal choice is", a.signal_search([0, 1, 2])

Initial beliefs:
Signal-action counts: {0: {10: 1}, 1: {9: 1, 10: 1}, 2: {8: 1, 9: 1, 10: 1}}
Prior beliefs: {0: {10: 1.0}, 1: {9: 0.3333333333333333, 10: 0.6666666666666666}, 2: {8: 0.3333333333333333, 9: 0.3333333333333333, 10: 0.3333333333333333}}
Risk attached to light signal: -10.0
Risk attached to moderate signal: -9.66666666667
Risk attached to heavy signal: -9.0
Signal choice is (0, -10.0)
After three games:
Signal-action counts: {0: {10: 3.0}, 1: {9: 2.0, 10: 1}, 2: {8: 1, 9: 1, 10: 1}}
Beliefs: {0: {10: 1.0}, 1: {9: 0.6666666666666666, 10: 0.3333333333333333}, 2: {8: 0.3333333333333333, 9: 0.3333333333333333, 10: 0.3333333333333333}}
Risk attached to light signal: -10.0
Risk attached to moderate signal: -9.33333333333
Risk attached to heavy signal: -9.0
Signal choice is (0, -10.0)


## Bayesian Risk Minimisation

In [21]:
a, b = make_agents(BayesianSignaller)
print a.signal_matches, a.type_matches
print "Risk attached to light signal:", a.risk(0)
print "Risk attached to moderate signal:", a.risk(1)
print "Risk attached to heavy signal:", a.risk(2)
print "Signal choice is", a.signal_search([0, 1, 2])
play_games(a, b)
print a.signal_matches, a.type_matches
a.update_beliefs()
print "Risk attached to light signal:", a.risk(0)
print "Risk attached to moderate signal:", a.risk(1)
print "Risk attached to heavy signal:", a.risk(2)
print "Signal choice is", a.signal_search([0, 1, 2])

{0: 0.0, 1: 0.0, 2: 0.0} {0: 0.0, 1: 0.0, 2: 0.0}
Risk attached to light signal: -10.0
Risk attached to moderate signal: -9.66666666667
Risk attached to heavy signal: -9.0
Signal choice is (0, -9.999999999999998)
{0: 2.0, 1: 1.0, 2: 0.0} {0: 0.0, 1: 3.0, 2: 0.0}
Risk attached to light signal: -10.0
Risk attached to moderate signal: -9.83333333333
Risk attached to heavy signal: -9.0
Signal choice is (0, -9.999999999999998)


## CPT Example

In [14]:
a, b = make_agents(ProspectTheorySignaller)
play_games(a, b)
a.update_beliefs()

In [20]:
prospects = a.collect_prospects(0)
print prospects
a.cpt_value(prospects)

[(10, 0.5), (10, 0.16666666666666666), (10, 0.125), (10, 0.125), (10, 0.041666666666666664), (10, 0.041666666666666664)]


7.585775750291834

In [17]:
prospects = a.collect_prospects(1)
print prospects
a.cpt_value(prospects)

[(10, 0.1111111111111111), (10, 0.1111111111111111), (10, 0.05555555555555555), (10, 0.05555555555555555), (9, 0.4444444444444444), (9, 0.2222222222222222)]


7.139727151083673

In [18]:
prospects = a.collect_prospects(2)
print prospects
a.cpt_value(prospects)

[(10, 0.08333333333333333), (10, 0.08333333333333333), (9, 0.08333333333333333), (9, 0.08333333333333333), (8, 0.3333333333333333), (8, 0.3333333333333333)]


6.622393760235688

In [4]:
a.get_memory()

(29, [(1, 0, 0, 10), (1, 0, 0, 10), (1, 1, 0, 9)])

In [5]:
a.payoff_count

{0: {10: 3.0}, 1: {9: 2.0, 10: 1}, 2: {8: 1, 9: 1, 10: 1}}

In [6]:
a.type_matches

{0: 0.0, 1: 0.0, 2: 0.0}

In [7]:
a.signal_matches

{0: 2.0, 1: 1.0, 2: 0.0}

In [11]:
pt, signal, response, payoff = a.get_memory()[1][0]
tmp_signaller = type(a)(player_type=pt)
a.exogenous_update(signal, response, tmp_signaller, payoff, midwife_type=pt)

0 sharing_lexicographic 10 1 1.0 0
Amending 3.0
Changed to 4.0


In [14]:
a.payoff_count

{0: {10: 4.0}, 1: {9: 2.0, 10: 1}, 2: {8: 1, 9: 1, 10: 1}}

In [10]:
a.exogenous

[(1, 0, 0, 10)]

In [12]:
a.get_memory()

(29, [(1, 0, 0, 10), (1, 0, 0, 10), (1, 1, 0, 9)])